# 人工知能のための数学最終レポート

*M21W0B09_入谷雄介*

***
## コンセプト
自宅の家賃が妥当なのかどうか調査するため、最寄り駅の賃貸物件の家賃から学習モデルを作成。<br>
散布図と線形回帰のプロットに自宅を被せて表示するプログラムを作成。<br>
基本的には「人工知能のための数学」の講義内で教授された記法を使う。

***
## 仮説
賃貸サイトを見ている限りでは割安であると予想している。

In [60]:
import numpy as np
import matplotlib.pyplot as plt


#### 賃貸住宅の基礎データ
※横軸 = (駅徒歩 / 2) + 築年数<br>

In [61]:
def generatePredictModel():
    with open('house_price.csv', 'r') as fd:
        feature_names = np.array(fd.read().split('\n')[0].split(','))

    csv_data = np.loadtxt('house_price.csv', delimiter=',', skiprows=1)

    learning_data = csv_data[:, np.logical_or(feature_names == 'FarFromStation', feature_names == 'Age')]
    learning_data = np.array([[i[0] / 2 , i[1]] for i in learning_data])

    true_data = csv_data[:, feature_names == 'TotalPrice']
    true_data = np.array([i[0] for i in true_data])

    return learning_data, true_data

In [62]:
# 駅徒歩 + 築年数　と　家賃を返してもらう
x1, yt = generatePredictModel()

x_mean = np.mean(x1)
x_std = np.std(x1)

x3 = (x1 - x_mean) / x_std


x_data = np.insert(x1, 0, 1.0, axis=1)
y_data = np.array([0 for i in range(len(x_data))])

In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=0.7, test_size=0.3,
    random_state=123)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1527, 3) (655, 3) (1527,) (655,)


In [64]:
x = x_train
yt = y_train

print(x)
print(yt)

[[ 1.   5.  43. ]
 [ 1.   5.  36. ]
 [ 1.   5.  36. ]
 ...
 [ 1.   2.5 36. ]
 [ 1.  17.5 26. ]
 [ 1.   4.  34. ]]
[0 0 0 ... 0 0 0]


#### シグモイド関数と予測関数の定義

In [65]:
# シグモイド関数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 予測値の計算
def pred(x, w):
    return sigmoid(x @ w)


#### 評価関数の定義

In [66]:
# 損失関数(交差エントロピー関数)
def cross_entropy(yt, yp):
    # 交差エントロピーの計算(この段階ではベクトル)
    ce1 = -(yt * np.log(yp) + (1 - yt) * np.log(1 - yp))
    # 交差エントロピーベクトルの平均値を計算
    return(np.mean(ce1))

# 予測結果の確率値から 0 or 1 を判断する関数
def classify(y):
    return np.where(y < 0.5, 0, 1)


# モデルの評価を行う関数
from sklearn.metrics import accuracy_score
def evaluate(xt, yt, w):

    # 予測値の計算
    yp = pred(xt, w)

    # 損失関数値の計算
    loss = cross_entropy(yt, yp)

    # 予測値(確率値)を0または1に変換
    yp_b = classify(yp)

    #精度の算出
    score = accuracy_score(yt, yp_b)
    return loss, score


In [67]:
# 初期化処理

# データ系列総数
# 次の行を正しく実装します
M  = x.shape[0]

# 入力データ次元数(ダミー変数を含む)
# 次の行を正しく実装します
D = x.shape[1]

# 繰り返し回数
iters = 2000

# 学習率
alpha = 0.01

# 初期値
np.random.seed(seed=100)
w = np.random.randn(D)

# 評価結果記録用 (損失関数と精度)
history = np.zeros((0,3))

In [68]:

# 繰り返しループ

for k in range(iters+1):

    # 予測値の計算 (7.8.1)
    yp = pred(x, w)

    # 誤差の計算 (7.8.2)
    yd = yp - yt

    # 勾配降下法の実装 (7.8.4)
    w = w - alpha * (x.T @ yd) / M

    # 学習曲線描画用データの計算、保存
    if k % 100 == 0:
        # 損失関数値の計算 (7.6.1)
        loss, score = evaluate(x_test, y_test, w)
        # 計算結果の記録
        history = np.vstack((history, np.array([k, loss, score])))
        # 画面表示
        # print( "iter = %d  loss = %f score = %f"
        #     % (k, loss, score))


<ipython-input-66-ddca02d95158>:4: RuntimeWarning: divide by zero encountered in log
  ce1 = -(yt * np.log(yp) + (1 - yt) * np.log(1 - yp))


In [69]:
# 予測値、正解値、誤差の表示(最初の10要素)

print('予測値:', yp[:10])
print('実測値', yt[:10])
print('誤差', yd[:10])


予測値: [4.86399872e-07 3.80233840e-06 3.80233840e-06 4.81925053e-05
 3.70668455e-05 8.71270297e-06 1.50384007e-05 1.43406549e-06
 3.39640076e-03 2.19277862e-05]
実測値 [0 0 0 0 0 0 0 0 0 0]
誤差 [4.86399872e-07 3.80233840e-06 3.80233840e-06 4.81925053e-05
 3.70668455e-05 8.71270297e-06 1.50384007e-05 1.43406549e-06
 3.39640076e-03 2.19277862e-05]


#### 散布図と回帰直線の同時表示

In [70]:
# 回帰直線描画用の座標値計算

# xの最小値、最大値の座標を計算
xl = np.array([[1, x3.min()],[1, x3.max()]])

# 対応するyの予測値を計算
yl = pred(xl, w)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

#### 我が家の値を正規化

In [ ]:
# 我が家の値生成
my_home_x = ((13.0 / 2.0) + 29.0 - x_mean) / x_std
my_home_y = 8.3


#### プロット表示

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(x3, yt, s=10, c='b')
plt.scatter(my_home_x, my_home_y, s=100, c='r', marker="*")
plt.xlabel('(FarFromStation / 2) + Age', fontsize=14)
plt.ylabel('TotalPrice', fontsize=14)
plt.plot(xl[:,1], yl, c='k')
plt.show()

#### 横軸について
駅徒歩と築年数を単純に足すだけだとinputとしては確実に誤っていると思われる。<br>
例えば、駅徒歩50分の新築物件のような場合、新築であるというのがウリなので駅徒歩が家賃の現象に影響することは無いと予想される。<br>
そこで、定量的に家賃に影響する築年数はいじらず、駅徒歩を半分にして異常値の丸め込みを行った。


### ToDo
横軸の(駅徒歩 / 2)についてはもう少し検討の余地があると思われる。<br>
また、地区や周辺の建物など値を増やしていくと精度は上がっていくはず。

### 結果
回帰直線よりも上に位置しているため、このエリア内では多少割高な家賃であると言える。<br>
築年数や駅徒歩を加味すると多少割安な印象があったのだが、個人的にはややがっかりな結果となった。<br>
今後転居する際などはこのアプリを用いて試算するのも1つ良いかもしれない。